<a href="https://colab.research.google.com/github/lijin-durairaj-code-mode/codes/blob/main/text2SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### package installation

In [ ]:
!pip install --upgrade pip setuptools wheel
!apt update && apt install -y build-essential cmake libopenblas-dev
!pip install llama-cpp-python

!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1
!pip3 install huggingface-hub
!pip3 install accelerate peft bitsandbytes transformers trl
!pip install nltk

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
33 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
 !pip install optuna==2.10.0

### import library

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from peft import AutoPeftModelForCausalLM, PeftModel
import torch
import numpy as np

#NLTK
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

import os
from google.colab import userdata

In [ ]:
import optuna

### setting google colab configuration

In [ ]:
os.environ['HUGGINGFACE_TOKEN']=userdata.get('hugging_face_token')

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

### downloading model & dataset

In [ ]:
# model_name='TheBloke/tinyLlama-1.1B-Chat-v1.0-GGUF'
# model_file='tinyllama-1.1b-chat-v1.0.Q8_0.gguf'

# model_path=hf_hub_download(model_name,filename=model_file)

# llm=Llama(
#     model_path=model_path,
#     n_ctx=512,
#     n_threads=8,
#     n_gpu_layers=40
# )

In [ ]:
dataset_id='b-mc2/sql-create-context'
data=load_dataset(dataset_id,split='train')
df=data.to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

### functionality

In [ ]:
def chat_template_for_training(context,answer,question):
  template=f'''/
  <|im_start|>user
  given the context, generate an SQL query for the following question
  context:{context}
  question:{question}
  <|im_end|>

  <|im_start|> assistant
  {answer}
  <|im_end|>
  '''
  template = "\n".join([line.lstrip() for line in template.splitlines()])
  return template

In [ ]:
df["text"] = df.apply(lambda x: chat_template_for_training(x["context"],
x["answer"], x["question"]), axis=1)

formatted_data=Dataset.from_pandas(df)
_data=formatted_data.train_test_split(train_size=0.75,test_size=0.25)

In [ ]:
def compute_metrics(eval_pred):
  print('compute metrics is called !')
  preds,labels=eval_pred
  pred_texts=tokenizer.batch_decode(preds,skip_special_tokens=True)
  label_texts=tokenizer.batch_decode(labels,skip_special_tokens=True)
  bleu_scores=[]
  smooth_func=SmoothingFunction().method1

  for pred,label in zip(pred_texts,label_texts):
    pred_tokens=pred.split()
    label_tokens=[label.split()]

    score=sentence_bleu(label_tokens,pred_tokens,smoothing_function=smooth_func)
    bleu_scores.append(score)
  print(np.mean(bleu_scores)*100)
  return {'bleu_score':np.mean(bleu_scores)*100}


### fine tuning the model

In [ ]:
model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer=AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token=tokenizer.eos_token

bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=True
)

model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto'
)

# Disable cache to improve training speed.
model.config.use_cache = False

# Set the temperature for pretraining to 1.
model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

###### config

In [ ]:
# Define the PEFT configuration.
peft_LORA_config = LoraConfig(
    # Set the rank of the LoRA projection matrix.
    r=8,
    # Set the alpha parameter for the LoRA projection matrix.
    lora_alpha=16,
    # Set the dropout rate for the LoRA projection matrix.
    lora_dropout=0.05,
    # Set the bias term to "none".
    bias="none",
    # Set the task type to "CAUSAL_LM".
    task_type="CAUSAL_LM"
)

#define SFT Config
training_args=SFTConfig(
      dataset_text_field='text',
      output_dir="tinyllama-sqllm-v1",
      per_device_train_batch_size=6,
      per_device_eval_batch_size=6,
      # Set the number of gradient accumulation steps.
      gradient_accumulation_steps=2,
      # Set the optimizer to use.
      optim="paged_adamw_32bit",
      packing=False,
      # Set the learning rate.
      learning_rate=2e-4,
      # Set the learning rate scheduler type.
      lr_scheduler_type="cosine",
      # Set the save strategy.
      save_strategy="epoch",
      # Set the logging steps.
      logging_steps=10,
      num_train_epochs=2,
      # Set the maximum number of training steps.
      max_steps=100,
      # Enable fp16 training.
      fp16=True,
      max_seq_length=1024,
      warmup_steps=100
)

In [ ]:
# _dummy_training=formatted_data.select(range(1000))
# _dummy_training_data=_dummy_training.train_test_split(test_size=0.25)

###### train

In [ ]:
trainer = SFTTrainer(
    model=model,
    peft_config=peft_LORA_config,
    train_dataset=_data['train'],
    eval_dataset=_data['test'],
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-12-1435d47c2250>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/750 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/750 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/750 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/750 [00:00<?, ? examples/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: durairaj-lijin (durairaj-lijin-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,2.205500
20,1.479800
30,0.968100
40,0.767000
50,0.661500
60,0.650600
70,0.633500
80,0.603100
90,0.627900
100,0.608900


TrainOutput(global_step=100, training_loss=0.9205771732330322, metrics={'train_runtime': 139.507, 'train_samples_per_second': 8.602, 'train_steps_per_second': 0.717, 'total_flos': 1323322667974656.0, 'train_loss': 0.9205771732330322})

## hyper parameter tuning

In [ ]:
def objectives(trail):
  try:
    learning_rate=trail.suggest_float('learning_rate',1e-5,2e-5,log=True)
    gradient_accumulation=trail.suggest_int('gradient_accumulation',1,10)
    warmup_steps=trail.suggest_int('warmup_step',300,1000)
    dropout_layer=trail.suggest_int('dropout_layer',0.001,0.5)
    size_per_batch=trail.suggest_int('training_size',1,50)
    num_epochs=trail.suggest_int('total_epochs',1,100)

    peft_LORA_config = LoraConfig(
      # Set the rank of the LoRA projection matrix.
      r=8,
      # Set the alpha parameter for the LoRA projection matrix.
      lora_alpha=16,
      # Set the dropout rate for the LoRA projection matrix.
      lora_dropout=dropout_layer,
      # Set the bias term to "none".
      bias="none",
      # Set the task type to "CAUSAL_LM".
      task_type="CAUSAL_LM"
    )

    training_args=SFTConfig(
    dataset_text_field='text',
    output_dir="tinyllama-sqllm-v1",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    # Set the number of gradient accumulation steps.
    gradient_accumulation_steps=gradient_accumulation,
    # Set the optimizer to use.
    optim="paged_adamw_32bit",
    packing=False,
    # Set the learning rate.
    learning_rate=learning_rate,
    # Set the learning rate scheduler type.
    lr_scheduler_type="cosine",
    # Set the save strategy.
    save_strategy="epoch",
    # Set the logging steps.
    logging_steps=10,
    num_train_epochs=num_epochs,
    # Set the maximum number of training steps.
    max_steps=100,
    # Enable fp16 training.
    fp16=True,
    max_seq_length=1024,
    warmup_steps=warmup_steps
    )

    trainer = SFTTrainer(
    model=model,
    peft_config=peft_LORA_config,
    train_dataset=_data['train'],
    eval_dataset=_data['test'],
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    )

    training=trainer.train()
    return training['Training Loss']

  except Exception as ex:
    print(ex)

In [ ]:
# torch.cuda.empty_cache()
study=optuna.create_study(direction='maximize')
study.optimize(objectives,n_trials=50)

[I 2025-02-24 10:16:55,051] A new study created in memory with name: no-name-d9d8318d-5b7f-48a0-9bf7-5eb63e82c4c1
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:560: UserWarning: The distribution is specified by [0.001, 0.5] and step=1, but the range is not divisible by `step`. It will be replaced by [0.001, 0.001].
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-13-b7bb42b55081>:50: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML:   0%|          | 0/58932 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/58932 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/58932 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/58932 [00:00<?, ? examples/s]

...

Applying chat template to eval dataset:   0%|          | 0/19645 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/19645 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/19645 [00:00<?, ? examples/s]

Step,Training Loss
10,2.874200
20,2.860800
30,2.861100
40,2.870700
50,2.878000


Step,Training Loss
10,2.874200
20,2.860800
30,2.861100
40,2.870700
50,2.878000


In [ ]:
# trainer.evaluate()
# torch.cuda.empty_cache()
study.best_params

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.17 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.89 GiB is free. Process 6697 has 11.85 GiB memory in use. Of the allocated memory 10.24 GiB is allocated by PyTorch, and 1.47 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## load the trained model

In [ ]:


# # Load the pre-trained model.
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16,
#     load_in_8bit=False,
#     device_map="auto",
#     trust_remote_code=True
# )

# # Load the PEFT model from a checkpoint.
# model_path = "/content/tinyllama-sqllm-v1/checkpoint-500"
# peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

# # Wrap the model with the PEFT model.
# model = peft_model.merge_and_unload()

In [ ]:
# def chat_template(question, context):
#     """
#     Creates a chat template for the Llama model.

#     Args:
#         question: The question to be answered.
#         context: The context information to be used for generating the answer.

#     Returns:
#         A string containing the chat template.
#     """

#     template = f"""\
#     <|im_start|>user
#     Given the context, generate an SQL query for the following question
#     context:{context}
#     question:{question}
#     <|im_end|>
#     <|im_start|>assistant
#     """
#     # Remove any leading whitespace characters from each line in the template.
#     template = "\n".join([line.lstrip() for line in template.splitlines()])
#     return template

In [ ]:
# question = "get me details of employee from the country Uganda"
# context = '''CREATE TABLE [dbo].[employee-list-waila](
# 	[Salutation] [nvarchar](50) NOT NULL,
# 	[Initial] [nvarchar](50) NULL,
# 	[User_Id] [nvarchar](50) NOT NULL,
# 	[Name] [nvarchar](50) NOT NULL,
# 	[Employee_Id] [nvarchar](50) NOT NULL,
# 	[Country] [nvarchar](50) NOT NULL,
# 	[Supervisor_Manager] [nvarchar](50) NULL,
# 	[Contracting_Company] [nvarchar](50) NULL,
# 	[Primary_Industry] [nvarchar](100) NULL,
# 	[Secondary_Industry] [nvarchar](100) NULL,
# 	[Sector] [nvarchar](50) NULL,
# 	[Expertise] [nvarchar](50) NULL,
# 	[Industry_Role] [nvarchar](50) NULL,
# 	[Designation] [nvarchar](50) NULL,
# 	[Grade] [nvarchar](50) NOT NULL,
# 	[Target_Chargeability] [tinyint] NULL,
# 	[Charge_Out_Rate] [float] NULL,
# 	[Last_Promotion_Date] [date] NULL,
# 	[Last_Promotional_Level] [nvarchar](50) NULL,
# 	[Job_Title] [nvarchar](50) NULL,
# 	[Professional_Since] [nvarchar](50) NOT NULL,
# 	[Hired_Date] [date] NOT NULL,
# 	[Relevant] [nvarchar](1) NULL,
# 	[Employee_Sponser] [bit] NOT NULL,
# 	[Job_Description] [nvarchar](1) NULL,
# 	[Emergency_Contact_Name] [nvarchar](50) NULL,
# 	[Emergency_Contact_Number] [float] NULL,
# 	[Regional_Supervisor] [nvarchar](50) NULL,
# 	[Office_Supervisor] [nvarchar](50) NULL,
# 	[Engagement_Supervisor] [nvarchar](50) NULL
# )'''
# prompt = chat_template(question,context)

# # Encode the prompt.
# inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# # Generate the output.
# output = model.generate(**inputs, max_new_tokens=512)

# # Decode the output.
# text = tokenizer.decode(output[0], skip_special_tokens=True)

# # Print the generated SQL query.
# print(text)

In [ ]:
from google.colab import files

In [ ]:
os.getenv('HF_TOKEN')
os.environ['HF_TOKEN']=userdata.get('hf_token')

### saving the model

In [ ]:
# from huggingface_hub import HfApi

# api = HfApi()
# # api.create_repo("LijinDurairaj/tinyLlama-text2SQL", repo_type="model")

# api.upload_folder(
#     folder_path=model_path,
#     repo_id="LijinDurairaj/tinyLlama-text2SQL",
#     repo_type="model"
# )